# Generate an image collection from a stack of netcdfs

(1) Convert netcdf files to compressed tiffs

In [ ]:
%%bash

src_path="/media/sami/srifai-ssd/data-ssd/tropomi-conus"
dst_path="/media/sami/srifai-ssd/data-ssd/tropomi-sif-tif"
files=`find $src_path -type f -name "*.nc" -printf "%f\n"`


# run gdal_translate to convert to tif
for f in $files
 do 
    echo $f
    new_f="${f%.*}.tif"
    gdal_translate -projwin_srs EPSG:4326 -a_srs EPSG:4326 -of COG -co BLOCKSIZE=512 -co TILING_SCHEME=CUSTOM -co "COMPRESS=DEFLATE" -co "NUM_THREADS=12" netCDF:$src_path/$f:SIF $dst_path/$new_f
#   gdalwarp -t_srs EPSG:4326 -tr 0.05 0.05 -r average -multi $path/$f /home/sami/scratch/frac_cover/$f &
 done

(2) Upload the files to a google cloud storage bucket

In [ ]:
%%bash
src_path="/media/sami/srifai-ssd/data-ssd/tropomi-conus"
files=`find $src_path -type f -name "*.nc" -printf "%f\n"`

for f in $files
 do
     new_f="${f%.*}"
     echo $new_f.tif
     d="${new_f//[!0-9]/}"
     date=${d:0:4}-${d:4:2}-${d:6:7}
     earthengine upload image --asset_id=users/srifai/tropomi-sif/$new_f gs://tropomi-sif-conus/tropomi-sif-tif/$new_f.tif
#      earthengine asset set --time_start $date users/srifai/tropomi-sif/$new_f
 done

(3) Add the date to each image

In [ ]:
%%bash
src_path="/media/sami/srifai-ssd/data-ssd/tropomi-conus"
files=`find $src_path -type f -name "*.nc" -printf "%f\n"`

for f in $files
 do
     new_f="${f%.*}"
     echo $new_f.tif
     d="${new_f//[!0-9]/}"
     date=${d:0:4}-${d:4:2}-${d:6:7}
     earthengine asset set --time_start $date users/srifai/tropomi-sif/$new_f
 done